## 3.6 UserDict 상속하기
- `UserDict`은 `dict`를 상속하지 않고, 내부에 실제 항목을 담고 있는 `data`라고 하는 `dict` 객체를 갖고 있다.
- `UserDict`이 `MutableMapping`을 상속한다. 따라서 `MutableMapping`과 `Mapping`의 모든 기능을 가지게 된다.
- `MutableMapping`의 유용한 메서드들
  - `update()`: 
  - `get()`:


In [1]:
# 예제 3-8 삽입, 갱신, 조회할 때 비문자열 키를 항상 문자열로 변환하는 StrKeyDict
import collections


class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [9]:
mapping = StrKeyDict()
mapping[3] = 'value'

print(mapping[3])
print(mapping['3'])
print(3 in mapping)

value
value
True


## 3.7 불변 매핑
- 표준 라이브러리의 매핑형은 모두 가변형이지만, 사용자가 매핑을 변경하지 못하도록 보장하고 싶은 경우 사용
- `types` 모듈의 `MappingProxyType` 클래스를 사용하면, 동적인 뷰는 제공하지만 읽기 전용인 `mappingproxy` 객체를 반환한다.
  - 원래 겍체가 변경되면 `mappingproxy`에 반영되지만, `mappingproxy`를 변경할 수는 없다.
  

In [11]:
from types import MappingProxyType


d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [12]:
d_proxy[1]

'A'

In [13]:
d_proxy[2] = 'B'  # d_proxy 객체는 불변형

TypeError: 'mappingproxy' object does not support item assignment

In [15]:
d[2] = 'B'  # 원래 객체는 가변형
d_proxy  # d_proxy 객체에도 반영된다.

mappingproxy({1: 'A', 2: 'B'})

In [16]:
d_proxy[2]

'B'

## 3.8 집합 이론
- `set`과 이의 불변형 버전인 `frozenset`이 있다.
- 집합 요소는 반드시 해시할 수 있어야 한다. `set`은 해시 가능하지 않지만, `frozenset`은 해시 가능하다.

### 3.8.1 집합 리터럴
- `set` 객체를 `{item1, item2, ..., item3}`으로 만들 수 있다. 
- 단, 공집합: `set()`을 사용해서 만들어야 한다. `{}`는 딕셔너리 객체를 만들게 된다.


In [22]:
s = {1}
print("type of s: ", type(s))
print("s: ", s)

s.pop()
print("s after pop(): ", s)

type of s:  <class 'set'>
s:  {1}
s after pop():  set()


In [21]:
s

{1}

- `set([item1, item2, ..., item3])`으로 만드는 것보다 빠르고 가독성 좋다.
- 집합명 검색 -> 리스트 생성 -> 생성자에 전달
- 반면, `{item1, item2, ..., item3}`는 `BUILD_SET`이라는 특수 바이트코드를 실행

In [23]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [24]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


`frozenset`에 대해서는 별도의 리터럴이 없기 때문에 언제나 생성자를 호출해서 생성해야 한다.


In [25]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

### 3.8.2 지능형 집합


In [38]:
# 예제 3-13 유니코드명 안에 'SIGN'이 들어 있는 단어를 가진 Latin-1 문자들의 집합 만들기
from unicodedata import name
{chr(i): name(chr(i), '') for i in range(32, 256)}

{' ': 'SPACE',
 '!': 'EXCLAMATION MARK',
 '"': 'QUOTATION MARK',
 '#': 'NUMBER SIGN',
 '$': 'DOLLAR SIGN',
 '%': 'PERCENT SIGN',
 '&': 'AMPERSAND',
 "'": 'APOSTROPHE',
 '(': 'LEFT PARENTHESIS',
 ')': 'RIGHT PARENTHESIS',
 '*': 'ASTERISK',
 '+': 'PLUS SIGN',
 ',': 'COMMA',
 '-': 'HYPHEN-MINUS',
 '.': 'FULL STOP',
 '/': 'SOLIDUS',
 '0': 'DIGIT ZERO',
 '1': 'DIGIT ONE',
 '2': 'DIGIT TWO',
 '3': 'DIGIT THREE',
 '4': 'DIGIT FOUR',
 '5': 'DIGIT FIVE',
 '6': 'DIGIT SIX',
 '7': 'DIGIT SEVEN',
 '8': 'DIGIT EIGHT',
 '9': 'DIGIT NINE',
 ':': 'COLON',
 ';': 'SEMICOLON',
 '<': 'LESS-THAN SIGN',
 '=': 'EQUALS SIGN',
 '>': 'GREATER-THAN SIGN',
 '?': 'QUESTION MARK',
 '@': 'COMMERCIAL AT',
 'A': 'LATIN CAPITAL LETTER A',
 'B': 'LATIN CAPITAL LETTER B',
 'C': 'LATIN CAPITAL LETTER C',
 'D': 'LATIN CAPITAL LETTER D',
 'E': 'LATIN CAPITAL LETTER E',
 'F': 'LATIN CAPITAL LETTER F',
 'G': 'LATIN CAPITAL LETTER G',
 'H': 'LATIN CAPITAL LETTER H',
 'I': 'LATIN CAPITAL LETTER I',
 'J': 'LATIN CAPITAL LETTER J

In [39]:
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### 3.8.3 집합 연산
- 책 참고

In [50]:
d = dict()
d[1] = 'A'
d[2] = 'B'

In [51]:
hash(d[1])

-5145490124865665313

In [52]:
hash(d[2])

-1584636571135021516

In [55]:
d[1].__hash__()

-5145490124865665313